# Creating custom estimator


**Reference**:

  * [Office Guide for creating custom estimator](https://www.tensorflow.org/get_started/custom_estimators)

To custom an estimator, there are two steps:

  * Define your model_fn().
  
  * Create your custom estimator.

## Preparation

In [2]:
import tensorflow as tf
import pandas as pd
import iris_data


# show package version
print("Tensorflow version: %s" % (tf.__version__))
print("Pandas version: %s" % (pd.__version__))

Tensorflow version: 1.7.0
Pandas version: 0.22.0


In this notebook, iris dataset is used as an example dataset. 

In [4]:
# Fetch iris data
iris_train, iris_test = iris_data.load_data()
iris_X_train, iris_y_train = iris_train
iris_X_test, iris_y_test = iris_test

Write an input function.

In [6]:
# training input function
def train_input_fn(features, labels, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()

## Define model_fn()

model_fn() is a function that gets invoked whenever someone calls the Estimator's train, evaluate, or predict methods. 

Recall that the signature for the model function looks like this:

    def my_model_fn(
       features, # This is batch_features from input_fn
       labels,   # This is batch_labels from input_fn
       mode,     # An instance of tf.estimator.ModeKeys, see below
       params):  # Additional configuration

`features` and `labels` are extraced from dataset returned from input_fn(). `mode` is a flag that represent "train", "evaluate" or "predict". `params` contains some parameters which are set at the time of estimator is created.

In [8]:
def my_model_fn(features, labels, mode, params):
    """DNN with three hidden layers, and dropout of 0.1 probability."""
    # Create three fully connected layers each layer having a dropout
    # probability of 0.1.
    net = tf.feature_column.input_layer(features, params['feature_columns'])
    for units in params['hidden_units']:
        net = tf.layers.dense(net, units=units, activation=tf.nn.relu)

    # Compute logits (1 per class).
    logits = tf.layers.dense(net, params['n_classes'], activation=None)

    # Compute predictions.
    predicted_classes = tf.argmax(logits, 1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_ids': predicted_classes[:, tf.newaxis],
            'probabilities': tf.nn.softmax(logits),
            'logits': logits,
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # Compute loss.
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Compute evaluation metrics.
    accuracy = tf.metrics.accuracy(labels=labels,
                                   predictions=predicted_classes,
                                   name='acc_op')
    metrics = {'accuracy': accuracy}
    tf.summary.scalar('accuracy', accuracy[1])

    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode, loss=loss, eval_metric_ops=metrics)

    # Create training op.
    assert mode == tf.estimator.ModeKeys.TRAIN

    optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

## Create Estimator

In [9]:
# feature columns
my_feature_columns = []
for key in iris_X_train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [10]:
# Build 2 hidden layer DNN with 10, 10 units respectively.
classifier = tf.estimator.Estimator(
    model_fn=my_model,
    params={
        'feature_columns': my_feature_columns,
        # Two hidden layers of 10 nodes each.
        'hidden_units': [10, 10],
        # The model must choose between 3 classes.
        'n_classes': 3,
    })

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/b6/4pljczw11w10f2tqrk_5gnp00000gn/T/tmpf7gpttdl', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1219733c8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/b6/4pljczw11w10f2tqrk_5gnp00000gn/T/tmpf7gpttdl', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1219733c8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Train model

In [12]:
# Train the Model.
classifier.train(
    input_fn=lambda:iris_data.train_input_fn(iris_X_train, iris_y_train, 10),
    steps=1000)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Create CheckpointSaverHook.


...

INFO:tensorflow:loss = 0.17981675, step = 901 (0.066 sec)


INFO:tensorflow:Saving checkpoints for 1000 into /var/folders/b6/4pljczw11w10f2tqrk_5gnp00000gn/T/tmpf7gpttdl/model.ckpt.


INFO:tensorflow:Saving checkpoints for 1000 into /var/folders/b6/4pljczw11w10f2tqrk_5gnp00000gn/T/tmpf7gpttdl/model.ckpt.


INFO:tensorflow:Loss for final step: 0.015158175.


INFO:tensorflow:Loss for final step: 0.015158175.


## Evaluate model

In [15]:
# Evaluate the model.
eval_result = classifier.evaluate(
    input_fn=lambda:iris_data.eval_input_fn(iris_X_test, iris_y_test,10))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Starting evaluation at 2018-04-02-04:39:10


...

INFO:tensorflow:Finished evaluation at 2018-04-02-04:39:10


INFO:tensorflow:Finished evaluation at 2018-04-02-04:39:10


INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.96666664, global_step = 1000, loss = 0.077173926


INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.96666664, global_step = 1000, loss = 0.077173926



Test set accuracy: 0.967



In [17]:
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

predictions = classifier.predict(
    input_fn=lambda:iris_data.eval_input_fn(predict_x,
                                            labels=None,
                                            batch_size=10))

for pred_dict, expec in zip(predictions, expected):
    template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print(template.format(iris_data.SPECIES[class_id],
                          100 * probability, expec))

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Graph was finalized.


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.



Prediction is "Setosa" (99.9%), expected "Setosa"

Prediction is "Versicolor" (97.8%), expected "Versicolor"

Prediction is "Virginica" (99.0%), expected "Virginica"
